<a href="https://colab.research.google.com/github/beercafeguy/PyMLLab/blob/master/grokking_dl/grokking_dl_chapter_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bag Of Words

In [1]:
import numpy as np


onehots = {

}

onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])


sentence = ['the','cat','sat']


x = onehots[sentence[0]] + onehots[sentence[1]] + onehots[sentence[2]]

print(x)

[1 1 0 1]


In [2]:
import sys

# https://github.com/iamtrask/Grokking-Deep-Learning/blob/master/reviews.txt
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

# https://github.com/iamtrask/Grokking-Deep-Learning/blob/master/labels.txt
f = open('labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [4]:
len(target_dataset)

25000

In [7]:
np.random.seed(1)

def sigmoid(x):
  return 1/(1+np.exp(-x))


def softmax(x):
  temp = np.exp(x)
  return temp / np.sum(temp, axis=1,keepdims=True)

alpha, iterations = (0.01, 2)
hidden_size = 100


weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1

correct,total = (0,0)

for iter in range(iterations):

  for i in range(len(input_dataset)-1000):
    X,y = (input_dataset[i],target_dataset[i])
    layer_1 = sigmoid(np.sum(weights_0_1[X],axis=0)) # embedding + sigmoid
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax


    layer_2_delta = layer_2 - y
    layer_1_delta = layer_2_delta.dot(weights_1_2.T) # backpropogation

    weights_0_1[X] -= layer_1_delta * alpha
    weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

    if(np.abs(layer_2_delta) < 0.5):
      correct += 1
    total += 1
  if(i % 10 == 9):
    progress = str(i/float(len(input_dataset)))
    sys.stdout.write('\rIter:'+str(iter) +' Progress:'+progress[2:4] +'.'+progress[4:6]+'% Training Accuracy:'+ str(correct/float(total)) + '%')
  print()

correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):
  X = input_dataset[i]
  y = target_dataset[i]

  layer_1 = sigmoid(np.sum(weights_0_1[X],axis=0))
  layer_2 = sigmoid(np.dot(layer_1,weights_1_2))

  if(np.abs(layer_2 - y) < 0.5):
    correct += 1
  total += 1
  print("Test Accuracy:" + str(correct / float(total)))

Iter:0 Progress:95.99% Training Accuracy:0.8343333333333334%
Iter:1 Progress:95.99% Training Accuracy:0.867875%
Test Accuracy:1.0
Test Accuracy:1.0
Test Accuracy:1.0
Test Accuracy:1.0
Test Accuracy:1.0
Test Accuracy:1.0
Test Accuracy:0.8571428571428571
Test Accuracy:0.875
Test Accuracy:0.8888888888888888
Test Accuracy:0.9
Test Accuracy:0.9090909090909091
Test Accuracy:0.9166666666666666
Test Accuracy:0.9230769230769231
Test Accuracy:0.9285714285714286
Test Accuracy:0.9333333333333333
Test Accuracy:0.9375
Test Accuracy:0.9411764705882353
Test Accuracy:0.9444444444444444
Test Accuracy:0.9473684210526315
Test Accuracy:0.9
Test Accuracy:0.9047619047619048
Test Accuracy:0.9090909090909091
Test Accuracy:0.9130434782608695
Test Accuracy:0.9166666666666666
Test Accuracy:0.92
Test Accuracy:0.9230769230769231
Test Accuracy:0.9259259259259259
Test Accuracy:0.9285714285714286
Test Accuracy:0.9310344827586207
Test Accuracy:0.9333333333333333
Test Accuracy:0.9032258064516129
Test Accuracy:0.90625
Te

In [28]:
def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

In [9]:
from collections import Counter
import math

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)


print(similar('beautiful'))
print(similar('terrible'))

[('beautiful', -0.0), ('delightful', -0.7123995137343394), ('liked', -0.7124842757581611), ('finest', -0.7302354337827286), ('very', -0.7345681490129226), ('tight', -0.7401456974725709), ('sweet', -0.7403528415484321), ('enjoyed', -0.7446972211777564), ('surprisingly', -0.7448805091284226), ('outstanding', -0.7463803283676221)]
[('terrible', -0.0), ('boring', -0.7139519664129819), ('avoid', -0.7421234059020628), ('dull', -0.7680812413384678), ('badly', -0.7682062690541611), ('disappointment', -0.7826869521780584), ('mess', -0.7827510853158978), ('redeeming', -0.7898800677191603), ('disappointing', -0.8207227190944237), ('lacks', -0.8402183675051119)]


# Fill in the blanks

In [10]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [11]:
alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def similar(target='beautiful'):
  target_index = word2index[target]

  scores = Counter()
  for word,index in word2index.items():
    raw_difference = weights_0_1[index] - (weights_0_1[target_index])
    squared_difference = raw_difference * raw_difference
    scores[word] = -math.sqrt(sum(squared_difference))
  return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
  for target_i in range(len(review)):

    # since it's really expensive to predict every vocabulary
    # we're only going to predict a random subset
    target_samples = [review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

    left_context = review[max(0,target_i-window):target_i]
    right_context = review[target_i+1:min(len(review),target_i+window)]

    layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
    layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
    layer_2_delta = layer_2 - layer_2_target
    layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

    weights_0_1[left_context+right_context] -= layer_1_delta * alpha
    weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha

  if(rev_i % 250 == 0):
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)) + "   " + str(similar('terrible')))
  sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)))
print(similar('terrible'))

Progress:0.30126

KeyboardInterrupt: 

In [12]:
def analogy(positive=['terrible','good'],negative=['bad']):

    norms = np.sum(weights_0_1 * weights_0_1,axis=1)
    norms.resize(norms.shape[0],1)

    normed_weights = weights_0_1 * norms

    query_vect = np.zeros(len(weights_0_1[0]))
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]

    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)[1:]